# 기본세팅

In [2]:
### 한글 폰트 설치
!apt-get install -y fonts-nanum
!fc-cache -fv
!rm ~/.cache/matplotlib -rf
# 설치 후 colab의 경우 Runtime > Restart session 필요

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-nanum is already the newest version (20200506-1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.
Font directories:
	/root/.local/share/fonts
	/usr/local/share/fonts
	/usr/share/fonts
	/root/.fonts
	/usr/share/fonts/truetype
	/usr/share/fonts/truetype/dejavu
	/usr/share/fonts/truetype/liberation
	/usr/share/fonts/truetype/nanum
/root/.local/share/fonts: skipping, no such directory
/usr/local/share/fonts: caching, new cache contents: 0 fonts, 0 dirs
/usr/share/fonts: caching, new cache contents: 0 fonts, 1 dirs
/usr/share/fonts/truetype: caching, new cache contents: 0 fonts, 3 dirs
/usr/share/fonts/truetype/dejavu: caching, new cache contents: 22 fonts, 0 dirs
/usr/share/fonts/truetype/liberation: caching, new cache contents: 16 fonts, 0 dirs
/usr/share/fonts/truetype/nanum: caching, new cache contents: 12 fonts, 0 dirs
/root/.fonts: skipping, no such directory
/usr/shar

In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rc('font', family='NanumBarunGothic') # 혹은 다른 설치한 Nanum 폰트 사용

In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import scipy
import scipy.stats as stats

# 서비스 방향 모색을 위한 분석
## 분석 목적 : 설정된 페르소나에 맞춤형 프로그램 제공을 위한 서비스 방향 모색
- 페르소나 : 희귀 신경계 질환을 앓고 있으며, 취사/청소 능력이 없는 서울 소재의 70대 독거 남성 

### 분석 방향
1) 전체 희귀질환 환자들의 니즈 파악
2) 신경계통 질환 환자들의 니즈 파악
3) 희귀 신경계 질환 환자들의 니즈 파악

#### 2. 신경계통 질환 환자들의 니즈 파악

##### dataset 불러오기
<  dataset : 전체 희귀질환 환자들 커뮤니티 >
- KMSS(한국다발성경화증협회)
- dcinside(디시인사이드)
- behcet(한국베체트환우협회)
- kfrd(한국희귀질환재단)
- kord((사)한국희귀. 난치성질환연합회)
- VonRecklinghausen(신경섬유종증환우)
- gowithms(MASTERING MS 다발성경화증 '행복한 동행')
- guillainbarre(길랑바레, 밀러피셔 증후군)
- neurofibromatosis(2형 신경섬유종)
- rheumatism(강인함(강직성척추염연합회))
- navercafe - 신경계 희귀질환

In [5]:
# mongo에서 db 불러오는 함수
import pymongo
def dbconnect(collection) :
    client=pymongo.MongoClient('mongodb://trainings.iptime.org',45003)
    db= client['data_analysis']
    collection = db[collection]
    data = collection.find()
    df = pd.DataFrame(data)
    return df

In [6]:
# 중복값 제거하는 함수(contents를 기준으로)
def drop_duplicates(df, contents) : 
    df.drop_duplicates(subset=contents, keep='first', inplace=True)
    df.reset_index(drop=True, inplace=True)
    return df

In [7]:
# 특정 열에서 list를 str으로 변환하는 함수
def list_join(row) :
    row = ' '.join(row)
    return row

In [8]:
# 특정 열에서 \n 삭제하는 함수
def remove_text(row):
    if pd.notnull(row):
        row = row.replace(r'\n', ' ')
    return row

In [9]:
# 리스트 값을 제거하는 함수
def remove_list(row):
    if isinstance(row, list):
        return ' '.join(row)
    else:
        return row

##### 데이터 전처리

###### KMSS

In [10]:
df_KMSS = dbconnect("KMSS")
df_KMSS

,_id,title,view,date,contents,reply
0,664162e6b6946f97c6276ef7,늦었습니다.,15회,2024.05.11 19:36:46,어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.\n오늘 아침에 퇴원을 하셔서...,이현호\n2024.05.13 09:17:41\n조인구님 안녕하세요.\n조인구님 글이...
1,664162ecb6946f97c6276ef8,Only Love - Nana Mouskouri: with Lyrics,14회,2024.05.07 16:13:59,상대에게 신뢰받는 5가지 방법 \n\n 1. 대화할 때 혼자만 이야기하지 말라....,이현호\n2024.05.08 10:25:51\n세상 살아가면서 대화가 잘 통하고 내...
2,664162f3b6946f97c6276ef9,하비(Harvey),18회,2024.05.03 16:19:36,https://youtu.be/q_TJlMUHfp4\n하비(Harvey) - 1부\...,구애경\n2024.05.03 19:05:44\nhttps://m.blog.naver...
3,664162f9b6946f97c6276efa,4월아 안녕~~.,20회,2024.04.30 15:06:42,"4월이 마침표를 찍는 날입니다.\n날씨가 변화무쌍했습니다.(추웠다 더웠다, 비오고 ...",서영임\n2024.04.30 15:33:46\n같은 아픔을 나누는 우리 모임에 모일...
4,66416300b6946f97c6276efb,4월을 마무리 짓는 날입니다.,16회,2024.04.30 11:12:18,오늘로서 4월이라는 달을 마무리 짓고 내일부터는 가정의 달이라고 하는 5월입니다.\...,이현호\n2024.04.30 11:56:18\n어머니가 편찮으셔서 걱정이 많이 되시...
...,...,...,...,...,...,...
884,664179b8b6946f97c627726b,건강상태 알려주는 신체의 아침신호,139회,2019.07.26 14:27:44,건강상태 알려주는 신체의 아침신호\n \n깨어나는 아침에 나타나는 증상들을 주의 깊...,
885,664179bfb6946f97c627726c,[제안] 국립 희귀질환센터 설립,190회,2019.07.22 10:33:20,국립 희귀질환센터 설립\n 제안이유\n 사람의 자유와 행복은 건강한 신체에 있습...,
886,664179c5b6946f97c627726d,치아관리 13계명,141회,2019.06.22 00:30:45,치아관리 13계명\n \n \n 건강한 이야말로 무덤까지 함께 갈 평생 건강의 필...,
887,664179cbb6946f97c627726e,고혈압에 좋은 음식과 차와 운동입니다.,159회,2019.06.15 20:14:53,"일차성고혈압에 대해 좋은 (음식, 차, 운동)\n 일차성고혈압이란 혈액이 혈관 벽에...",


In [11]:
# 중복데이터 확인
df_KMSS['contents'].value_counts()
# 중복데이터 없음

contents
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

In [12]:
df_KMSS.reset_index(drop=True, inplace=True)
df_KMSS

,_id,title,view,date,contents,reply
0,664162e6b6946f97c6276ef7,늦었습니다.,15회,2024.05.11 19:36:46,어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.\n오늘 아침에 퇴원을 하셔서...,이현호\n2024.05.13 09:17:41\n조인구님 안녕하세요.\n조인구님 글이...
1,664162ecb6946f97c6276ef8,Only Love - Nana Mouskouri: with Lyrics,14회,2024.05.07 16:13:59,상대에게 신뢰받는 5가지 방법 \n\n 1. 대화할 때 혼자만 이야기하지 말라....,이현호\n2024.05.08 10:25:51\n세상 살아가면서 대화가 잘 통하고 내...
2,664162f3b6946f97c6276ef9,하비(Harvey),18회,2024.05.03 16:19:36,https://youtu.be/q_TJlMUHfp4\n하비(Harvey) - 1부\...,구애경\n2024.05.03 19:05:44\nhttps://m.blog.naver...
3,664162f9b6946f97c6276efa,4월아 안녕~~.,20회,2024.04.30 15:06:42,"4월이 마침표를 찍는 날입니다.\n날씨가 변화무쌍했습니다.(추웠다 더웠다, 비오고 ...",서영임\n2024.04.30 15:33:46\n같은 아픔을 나누는 우리 모임에 모일...
4,66416300b6946f97c6276efb,4월을 마무리 짓는 날입니다.,16회,2024.04.30 11:12:18,오늘로서 4월이라는 달을 마무리 짓고 내일부터는 가정의 달이라고 하는 5월입니다.\...,이현호\n2024.04.30 11:56:18\n어머니가 편찮으셔서 걱정이 많이 되시...
...,...,...,...,...,...,...
884,664179b8b6946f97c627726b,건강상태 알려주는 신체의 아침신호,139회,2019.07.26 14:27:44,건강상태 알려주는 신체의 아침신호\n \n깨어나는 아침에 나타나는 증상들을 주의 깊...,
885,664179bfb6946f97c627726c,[제안] 국립 희귀질환센터 설립,190회,2019.07.22 10:33:20,국립 희귀질환센터 설립\n 제안이유\n 사람의 자유와 행복은 건강한 신체에 있습...,
886,664179c5b6946f97c627726d,치아관리 13계명,141회,2019.06.22 00:30:45,치아관리 13계명\n \n \n 건강한 이야말로 무덤까지 함께 갈 평생 건강의 필...,
887,664179cbb6946f97c627726e,고혈압에 좋은 음식과 차와 운동입니다.,159회,2019.06.15 20:14:53,"일차성고혈압에 대해 좋은 (음식, 차, 운동)\n 일차성고혈압이란 혈액이 혈관 벽에...",


In [13]:
# 각 컬럼별 전처리(제목)
df_KMSS['title']

0                                       늦었습니다.
1      Only Love - Nana Mouskouri: with Lyrics
2                                   하비(Harvey)
3                                    4월아 안녕~~.
4                             4월을 마무리 짓는 날입니다.
                        ...                   
884                         건강상태 알려주는 신체의 아침신호
885                          [제안] 국립 희귀질환센터 설립
886                                  치아관리 13계명
887                      고혈압에 좋은 음식과 차와 운동입니다.
888                  커피가 하루동안 인체에 미치는 영향이 무얼까?
Name: title, Length: 889, dtype: object

In [14]:
# 각 컬럼별 전처리(내용)
# 특정 열에서 \n 삭제
df_KMSS['contents'] = df_KMSS['contents'].str.replace('\n', '')

df_KMSS['contents']

0      어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.오늘 아침에 퇴원을 하셔서 집...
1       상대에게 신뢰받는 5가지 방법   1. 대화할 때 혼자만 이야기하지 말라.    ...
2      https://youtu.be/q_TJlMUHfp4하비(Harvey) - 1부htt...
3      4월이 마침표를 찍는 날입니다.날씨가 변화무쌍했습니다.(추웠다 더웠다, 비오고 흐린...
4      오늘로서 4월이라는 달을 마무리 짓고 내일부터는 가정의 달이라고 하는 5월입니다.우...
                             ...                        
884    건강상태 알려주는 신체의 아침신호 깨어나는 아침에 나타나는 증상들을 주의 깊게 살핀...
885     국립 희귀질환센터 설립  제안이유 사람의 자유와 행복은 건강한 신체에 있습니다. ...
886    치아관리 13계명    건강한 이야말로 무덤까지 함께 갈 평생 건강의 필수 동반자다...
887    일차성고혈압에 대해 좋은 (음식, 차, 운동) 일차성고혈압이란 혈액이 혈관 벽에 가...
888    대개의 사람들이 커피를 좋아한다.2015년에 시행한 여론조사에 따르면, 미국인 중 ...
Name: contents, Length: 889, dtype: object

In [15]:
# 각 컬럼별 전처리(댓글)
# 특정 열에서 \n 삭제
df_KMSS['reply'] = df_KMSS['reply'].str.replace('\n', '')
# 날짜와 고유명사 삭제
df_KMSS['reply'] = df_KMSS['reply'].str.replace(r'\b\w+\d{4}\.\d{2}\.\d{2}\s+\d{2}:\d{2}:\d{2}', '', regex=True)

df_KMSS['reply']

0      조인구님 안녕하세요.조인구님 글이 올라오지않아서 어머님의 건강을 걱정하고 있었습니다...
1      세상 살아가면서 대화가 잘 통하고 내 이야기 잘 경청해주고 공감해주는 사람이 제일 ...
2      https://m.blog.naver.com/PostView.naver?blogId...
3      같은 아픔을 나누는 우리 모임에 모일 수 없으면 찾아가야죠. 시간이 허락되면 동참하...
4      어머니가 편찮으셔서 걱정이 많이 되시겠어요.병원에 꽤 오래 계시네요.하루속히 쾌차하...
                             ...                        
884                                                     
885                                                     
886                                                     
887                                                     
888        흥미로운 글이네요💇잘 읽었습니다🙋✌커피 좋아하시는분들은 꼭 읽어봤으면 좋겠어요:)
Name: reply, Length: 889, dtype: object

In [16]:
# title, contents, reply 컬럼을 선택하여 합친 후 1개의 시리즈로 만들어서 list로 변환
KMSS_series = df_KMSS['title'] + ' ' + df_KMSS['contents']
KMSS_series.isna().sum()

0

In [17]:
KMSS_list = KMSS_series.tolist()
KMSS_list

["늦었습니다. 어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.오늘 아침에 퇴원을 하셔서 집에 와서 여러가지 일을 하다보니 글 올리는 것이 생각이 났습니다.;오늘은 우리가 많이 먹는 포도에 대한 효능과 부작용에 대한 글입니다.포도 효능 11가지한여름이 지나가고 가을이 올 때쯤 되면 한참 달콤한 과일을 먹을 수 있어서 좋은 거 같다. 달콤한 복숭아도 좋고, 새콤한 자두도 좋지만 한 알 한 알 톡톡 터지는 식감이 좋은 포도도 너무 좋아해요. 아이들 간식부터 어르신들 입맛까지 사로잡은 과일로 누구나 좋아하는 음식으로 손꼽힌다.7~8월이 제철이어서 무더운 여름에 맛있게 먹을 수 있는 과일입니다. 특유의 향이 좋고 과즙이 풍부해서 과일 자체로도 맛있지만 주스로 마시거나 건포도로 가공해서 먹기도 한다. 우리나라는 이 과일을 꽤 옛날부터 먹어왔습니다.포도 효능1. 혈압을 조절해 준다.포도는 심장 건강에 도움이 되는 한 부분으로 혈압에 긍정적인 영향을 미칩니다. 혈압은 동맥혈관 벽에 대항한 혈액의 압력을 말한다. 이 혈액의 압력이 여러 원인으로 인해 높아진 상태를 고혈압이라 하는데, 우리나라 성인 인구의 30%가량이 고혈압 증상이 있는 것으로 추정하고 있습니다. 포도에는 칼륨과 섬유질이 풍부해서 나트륨의 배출을 돕습니다. 나트륨이 고혈압을 유발하는 원인 중 한 가지입니다. 포도를 적정량 꾸준히 섭취하다 보면 나트륨으로 인한 고혈압의 위험을 낮출 수 있습니다.2. 치매를 예방해 준다.포도 씨의 폴리페놀 성분은 알츠하이머의 진전을 늦춰줍니다. 연구에 의하면 실험용 생쥐에게 포도 씨의 폴리페놀 추출물을 5개월 동안 투여한 결과, 기억력 감퇴에 영향을 주는 ‘아밀로이드 베타-56’이라 하는 신경독소 물질이 감소한 것으로 나타났습니다.3. 항암 효과를 볼 수 있다.포도에는 폴리페놀이라는 항산화 성분이 포함되어 있습니다. 이들은 산화방지제 역할을 하여 암 관리와과 세포 손상 및 보호에 도움을 줄 수 있습니다. 특히 적포도에는 레스베라트롤이라는 폴리페놀의 일종인 성분이 많이 함유

###### gowithms

In [18]:
df_gowithms = dbconnect("persona1_navercafe_gowithms")
df_gowithms

,_id,cafe,title,name,date,contents,review
0,66333f813982c9b53d975588,MASTERING MS 다발성경화증,소아 시신경 염증 (다발성경화증 유전 가능성),잔댕,2024.04.24. 17:14,안녕하세요.\n조카의 엄마가 다발성경화증을 앓았고.. (증상: 안구가 중심에서 바깥...,[다발성 경화증은 명확한 발병 요인이 없어서 그냥 유전적+환경적 요인이라고 알고는 ...
1,66333f873982c9b53d975589,MASTERING MS 다발성경화증,재발한걸까요?,동군용이,2024.04.20. 22:35,안녕하세요. 20년도에 다발성경화증 진단 받았습니다. 3일 전부터 오른쪽 팔이 근육...,"[혹시 모르니 일단 병원진료 받아보세요, 안녕하세요, 저도 복시로 첫 증상이 시작돼..."
2,66333f8d3982c9b53d97558a,MASTERING MS 다발성경화증,진단을 받기 직전 어떤 증상을 겪으셨나요?,유유 ll,2024.04.20. 19:44,안녕하세요\n29살 여자입니다\n최근 두통과 어지럼증이 심해져 신경과를 다니고 있어...,[안녕하세요. 저는 처음에 어지러움과 동시에 복시가 와서ㅜㅜ 단순히 눈 문제라고 생...
3,66333f933982c9b53d97558b,MASTERING MS 다발성경화증,"📰 희귀질환 최신 뉴스 모음 💚 4월 첫째, 둘째 주",레어노트,2024.04.16. 14:50,안녕하세요. 레어노트 팀입니다.\n오늘은 반가운 소식을 함께 들고 왔는데요.\n이제...,[안녕하세요. 저는 처음에 어지러움과 동시에 복시가 와서ㅜㅜ 단순히 눈 문제라고 생...
4,66333f9a3982c9b53d97558c,MASTERING MS 다발성경화증,텍피데라 팔 두드러기요…,진주88,2024.04.15. 22:00,여자인데 목 왼쪽팔 이제 얼굴까지\n이번에 재발하고 다발성경화증 진단받아서 텍피데라...,"[저는 두드러기가 배에서 시작했다가 점점 목 아래, 허벅지, 무릎까지 번지고 엄청나..."
...,...,...,...,...,...,...,...
468,66334aec3982c9b53d97575c,MASTERING MS 다발성경화증,도연이아빠입니다,도연아빠,2010.09.15. 17:43,다발성경화증과 담도폐쇄증을 알고 있는 도연이아빠입니다\n안녕하세요\n이제 가입했네요...,[안녕하세요 도연아빠님 ^^ 저희 카페에 등업은 따로 없습니다 ^^ 자주 오셔서 많...
469,66334af23982c9b53d97575d,MASTERING MS 다발성경화증,안녕하셔요~,rhdcl1,2010.09.15. 14:05,안녕하세요?\n저는 올1월에 다리에 감각이 떨어져 2월에 급하게 입원후 ..\n이런...,[안녕하세요! ^^ rhdcl1님의 긍정적인 마음을 보니 저도 힘이 나는 것 같습니...
470,66334af83982c9b53d97575e,MASTERING MS 다발성경화증,저도 인사드립니다,on_the_go,2010.09.14. 10:46,안녕하세요 저도 어제 가입하고 처음 인사 드립니다.\n새로 생긴 카페인줄 알았는데 ...,[안녕하세요 ^^ 앞으로 자주 방문하셔서 좋은 정보 많이 얻고 가세요 ~ on_th...
471,66334afe3982c9b53d97575f,MASTERING MS 다발성경화증,안녕하세요 ^^,밝은마음,2010.09.13. 10:34,안녕하세요\n다발성경화증에 관란 정보를 검색하다가 가입하게 되었습니다.\n이런 카페...,"[안녕하세요 반갑습니다 밝은마음님! 카페에서 자주 봬요 ^^, ☎비‥아‥그‥라 ⇒ ..."


In [19]:
# 중복값 제거
df_gowithms = drop_duplicates(df_gowithms, 'contents')

In [20]:
# 각 컬럼별 전처리(제목)
df_gowithms['title']

0          소아 시신경 염증 (다발성경화증 유전 가능성)
1                            재발한걸까요?
2            진단을 받기 직전 어떤 증상을 겪으셨나요?
3      📰 희귀질환 최신 뉴스 모음 💚 4월 첫째, 둘째 주
4                      텍피데라 팔 두드러기요…
                   ...              
463                         도연이아빠입니다
464                           안녕하셔요~
465                        저도 인사드립니다
466                         안녕하세요 ^^
467                         인사드려요!^^
Name: title, Length: 468, dtype: object

In [21]:
# '📰 희귀질환 최신 뉴스 모음 💚' 와 같은 정기적인 광고글 확인
# '희귀질환 최신 뉴스 모음'이라는 내용이 담겨있는 행(row) 삭제
df_gowithms = df_gowithms[~df_gowithms['title'].str.contains('희귀질환 최신 뉴스 모음')]

df_gowithms

,_id,cafe,title,name,date,contents,review
0,66333f813982c9b53d975588,MASTERING MS 다발성경화증,소아 시신경 염증 (다발성경화증 유전 가능성),잔댕,2024.04.24. 17:14,안녕하세요.\n조카의 엄마가 다발성경화증을 앓았고.. (증상: 안구가 중심에서 바깥...,[다발성 경화증은 명확한 발병 요인이 없어서 그냥 유전적+환경적 요인이라고 알고는 ...
1,66333f873982c9b53d975589,MASTERING MS 다발성경화증,재발한걸까요?,동군용이,2024.04.20. 22:35,안녕하세요. 20년도에 다발성경화증 진단 받았습니다. 3일 전부터 오른쪽 팔이 근육...,"[혹시 모르니 일단 병원진료 받아보세요, 안녕하세요, 저도 복시로 첫 증상이 시작돼..."
2,66333f8d3982c9b53d97558a,MASTERING MS 다발성경화증,진단을 받기 직전 어떤 증상을 겪으셨나요?,유유 ll,2024.04.20. 19:44,안녕하세요\n29살 여자입니다\n최근 두통과 어지럼증이 심해져 신경과를 다니고 있어...,[안녕하세요. 저는 처음에 어지러움과 동시에 복시가 와서ㅜㅜ 단순히 눈 문제라고 생...
4,66333f9a3982c9b53d97558c,MASTERING MS 다발성경화증,텍피데라 팔 두드러기요…,진주88,2024.04.15. 22:00,여자인데 목 왼쪽팔 이제 얼굴까지\n이번에 재발하고 다발성경화증 진단받아서 텍피데라...,"[저는 두드러기가 배에서 시작했다가 점점 목 아래, 허벅지, 무릎까지 번지고 엄청나..."
5,66333fa03982c9b53d97558d,MASTERING MS 다발성경화증,혹시 직장 바꾸신분들 어느업종으로 가셨나요?,09둘기,2024.04.04. 13:38,지금 회사에서 상담업무같은거를 맡고 있긴한데요\n저번에 재발한후 가끔씩 자주 까먹고...,[저는 보육교사 9년하고 발병하고 대학교 계약직원으로 (산학협력단이나 대학교 부설 ...
...,...,...,...,...,...,...,...
463,66334aec3982c9b53d97575c,MASTERING MS 다발성경화증,도연이아빠입니다,도연아빠,2010.09.15. 17:43,다발성경화증과 담도폐쇄증을 알고 있는 도연이아빠입니다\n안녕하세요\n이제 가입했네요...,[안녕하세요 도연아빠님 ^^ 저희 카페에 등업은 따로 없습니다 ^^ 자주 오셔서 많...
464,66334af23982c9b53d97575d,MASTERING MS 다발성경화증,안녕하셔요~,rhdcl1,2010.09.15. 14:05,안녕하세요?\n저는 올1월에 다리에 감각이 떨어져 2월에 급하게 입원후 ..\n이런...,[안녕하세요! ^^ rhdcl1님의 긍정적인 마음을 보니 저도 힘이 나는 것 같습니...
465,66334af83982c9b53d97575e,MASTERING MS 다발성경화증,저도 인사드립니다,on_the_go,2010.09.14. 10:46,안녕하세요 저도 어제 가입하고 처음 인사 드립니다.\n새로 생긴 카페인줄 알았는데 ...,[안녕하세요 ^^ 앞으로 자주 방문하셔서 좋은 정보 많이 얻고 가세요 ~ on_th...
466,66334afe3982c9b53d97575f,MASTERING MS 다발성경화증,안녕하세요 ^^,밝은마음,2010.09.13. 10:34,안녕하세요\n다발성경화증에 관란 정보를 검색하다가 가입하게 되었습니다.\n이런 카페...,"[안녕하세요 반갑습니다 밝은마음님! 카페에서 자주 봬요 ^^, ☎비‥아‥그‥라 ⇒ ..."


In [22]:
# 각 컬럼별 전처리(내용)
df_gowithms['contents']

0      안녕하세요.\n조카의 엄마가 다발성경화증을 앓았고.. (증상: 안구가 중심에서 바깥...
1      안녕하세요. 20년도에 다발성경화증 진단 받았습니다. 3일 전부터 오른쪽 팔이 근육...
2      안녕하세요\n29살 여자입니다\n최근 두통과 어지럼증이 심해져 신경과를 다니고 있어...
4      여자인데 목 왼쪽팔 이제 얼굴까지\n이번에 재발하고 다발성경화증 진단받아서 텍피데라...
5      지금 회사에서 상담업무같은거를 맡고 있긴한데요\n저번에 재발한후 가끔씩 자주 까먹고...
                             ...                        
463    다발성경화증과 담도폐쇄증을 알고 있는 도연이아빠입니다\n안녕하세요\n이제 가입했네요...
464    안녕하세요?\n저는 올1월에 다리에 감각이 떨어져 2월에 급하게 입원후 ..\n이런...
465    안녕하세요 저도 어제 가입하고 처음 인사 드립니다.\n새로 생긴 카페인줄 알았는데 ...
466    안녕하세요\n다발성경화증에 관란 정보를 검색하다가 가입하게 되었습니다.\n이런 카페...
467    안녕하세요~가양동에 살고 있는 30대 아줌마입니다^_^\n지인의 다발성경화증에 대한...
Name: contents, Length: 455, dtype: object

In [23]:
df_gowithms.loc[:,'contents'] = df_gowithms['contents'].apply(remove_text)
df_gowithms

,_id,cafe,title,name,date,contents,review
0,66333f813982c9b53d975588,MASTERING MS 다발성경화증,소아 시신경 염증 (다발성경화증 유전 가능성),잔댕,2024.04.24. 17:14,안녕하세요.\n조카의 엄마가 다발성경화증을 앓았고.. (증상: 안구가 중심에서 바깥...,[다발성 경화증은 명확한 발병 요인이 없어서 그냥 유전적+환경적 요인이라고 알고는 ...
1,66333f873982c9b53d975589,MASTERING MS 다발성경화증,재발한걸까요?,동군용이,2024.04.20. 22:35,안녕하세요. 20년도에 다발성경화증 진단 받았습니다. 3일 전부터 오른쪽 팔이 근육...,"[혹시 모르니 일단 병원진료 받아보세요, 안녕하세요, 저도 복시로 첫 증상이 시작돼..."
2,66333f8d3982c9b53d97558a,MASTERING MS 다발성경화증,진단을 받기 직전 어떤 증상을 겪으셨나요?,유유 ll,2024.04.20. 19:44,안녕하세요\n29살 여자입니다\n최근 두통과 어지럼증이 심해져 신경과를 다니고 있어...,[안녕하세요. 저는 처음에 어지러움과 동시에 복시가 와서ㅜㅜ 단순히 눈 문제라고 생...
4,66333f9a3982c9b53d97558c,MASTERING MS 다발성경화증,텍피데라 팔 두드러기요…,진주88,2024.04.15. 22:00,여자인데 목 왼쪽팔 이제 얼굴까지\n이번에 재발하고 다발성경화증 진단받아서 텍피데라...,"[저는 두드러기가 배에서 시작했다가 점점 목 아래, 허벅지, 무릎까지 번지고 엄청나..."
5,66333fa03982c9b53d97558d,MASTERING MS 다발성경화증,혹시 직장 바꾸신분들 어느업종으로 가셨나요?,09둘기,2024.04.04. 13:38,지금 회사에서 상담업무같은거를 맡고 있긴한데요\n저번에 재발한후 가끔씩 자주 까먹고...,[저는 보육교사 9년하고 발병하고 대학교 계약직원으로 (산학협력단이나 대학교 부설 ...
...,...,...,...,...,...,...,...
463,66334aec3982c9b53d97575c,MASTERING MS 다발성경화증,도연이아빠입니다,도연아빠,2010.09.15. 17:43,다발성경화증과 담도폐쇄증을 알고 있는 도연이아빠입니다\n안녕하세요\n이제 가입했네요...,[안녕하세요 도연아빠님 ^^ 저희 카페에 등업은 따로 없습니다 ^^ 자주 오셔서 많...
464,66334af23982c9b53d97575d,MASTERING MS 다발성경화증,안녕하셔요~,rhdcl1,2010.09.15. 14:05,안녕하세요?\n저는 올1월에 다리에 감각이 떨어져 2월에 급하게 입원후 ..\n이런...,[안녕하세요! ^^ rhdcl1님의 긍정적인 마음을 보니 저도 힘이 나는 것 같습니...
465,66334af83982c9b53d97575e,MASTERING MS 다발성경화증,저도 인사드립니다,on_the_go,2010.09.14. 10:46,안녕하세요 저도 어제 가입하고 처음 인사 드립니다.\n새로 생긴 카페인줄 알았는데 ...,[안녕하세요 ^^ 앞으로 자주 방문하셔서 좋은 정보 많이 얻고 가세요 ~ on_th...
466,66334afe3982c9b53d97575f,MASTERING MS 다발성경화증,안녕하세요 ^^,밝은마음,2010.09.13. 10:34,안녕하세요\n다발성경화증에 관란 정보를 검색하다가 가입하게 되었습니다.\n이런 카페...,"[안녕하세요 반갑습니다 밝은마음님! 카페에서 자주 봬요 ^^, ☎비‥아‥그‥라 ⇒ ..."


In [24]:
# 쓸데없는 광고글 drop
filtered_rows = df_gowithms[df_gowithms['contents'].str.contains('중고차수출')]
print(filtered_rows['contents'])

11    "노력 없이는 성취가 없다." - 사무엘 골드윈\n-중고차수출 가능 차종-\n/베르...
29    "승리는 노력과 열정의 결실이다." - 윌리엄 셰익스피어\n-중고차수출 가능 차종-...
Name: contents, dtype: object


In [25]:
# 쓸데없는 광고글 drop
df_gowithms = df_gowithms[~df_gowithms['contents'].str.contains('중고차수출')]
df_gowithms = df_gowithms[~df_gowithms['contents'].str.contains('레어노트')]
df_gowithms = df_gowithms[~df_gowithms['contents'].str.contains('PLEASE COME AND JOIN')]
df_gowithms = df_gowithms[~df_gowithms['contents'].str.contains('카지노')]
df_gowithms = df_gowithms[~df_gowithms['contents'].str.contains('shapetype')]
df_gowithms

,_id,cafe,title,name,date,contents,review
0,66333f813982c9b53d975588,MASTERING MS 다발성경화증,소아 시신경 염증 (다발성경화증 유전 가능성),잔댕,2024.04.24. 17:14,안녕하세요.\n조카의 엄마가 다발성경화증을 앓았고.. (증상: 안구가 중심에서 바깥...,[다발성 경화증은 명확한 발병 요인이 없어서 그냥 유전적+환경적 요인이라고 알고는 ...
1,66333f873982c9b53d975589,MASTERING MS 다발성경화증,재발한걸까요?,동군용이,2024.04.20. 22:35,안녕하세요. 20년도에 다발성경화증 진단 받았습니다. 3일 전부터 오른쪽 팔이 근육...,"[혹시 모르니 일단 병원진료 받아보세요, 안녕하세요, 저도 복시로 첫 증상이 시작돼..."
2,66333f8d3982c9b53d97558a,MASTERING MS 다발성경화증,진단을 받기 직전 어떤 증상을 겪으셨나요?,유유 ll,2024.04.20. 19:44,안녕하세요\n29살 여자입니다\n최근 두통과 어지럼증이 심해져 신경과를 다니고 있어...,[안녕하세요. 저는 처음에 어지러움과 동시에 복시가 와서ㅜㅜ 단순히 눈 문제라고 생...
4,66333f9a3982c9b53d97558c,MASTERING MS 다발성경화증,텍피데라 팔 두드러기요…,진주88,2024.04.15. 22:00,여자인데 목 왼쪽팔 이제 얼굴까지\n이번에 재발하고 다발성경화증 진단받아서 텍피데라...,"[저는 두드러기가 배에서 시작했다가 점점 목 아래, 허벅지, 무릎까지 번지고 엄청나..."
5,66333fa03982c9b53d97558d,MASTERING MS 다발성경화증,혹시 직장 바꾸신분들 어느업종으로 가셨나요?,09둘기,2024.04.04. 13:38,지금 회사에서 상담업무같은거를 맡고 있긴한데요\n저번에 재발한후 가끔씩 자주 까먹고...,[저는 보육교사 9년하고 발병하고 대학교 계약직원으로 (산학협력단이나 대학교 부설 ...
...,...,...,...,...,...,...,...
463,66334aec3982c9b53d97575c,MASTERING MS 다발성경화증,도연이아빠입니다,도연아빠,2010.09.15. 17:43,다발성경화증과 담도폐쇄증을 알고 있는 도연이아빠입니다\n안녕하세요\n이제 가입했네요...,[안녕하세요 도연아빠님 ^^ 저희 카페에 등업은 따로 없습니다 ^^ 자주 오셔서 많...
464,66334af23982c9b53d97575d,MASTERING MS 다발성경화증,안녕하셔요~,rhdcl1,2010.09.15. 14:05,안녕하세요?\n저는 올1월에 다리에 감각이 떨어져 2월에 급하게 입원후 ..\n이런...,[안녕하세요! ^^ rhdcl1님의 긍정적인 마음을 보니 저도 힘이 나는 것 같습니...
465,66334af83982c9b53d97575e,MASTERING MS 다발성경화증,저도 인사드립니다,on_the_go,2010.09.14. 10:46,안녕하세요 저도 어제 가입하고 처음 인사 드립니다.\n새로 생긴 카페인줄 알았는데 ...,[안녕하세요 ^^ 앞으로 자주 방문하셔서 좋은 정보 많이 얻고 가세요 ~ on_th...
466,66334afe3982c9b53d97575f,MASTERING MS 다발성경화증,안녕하세요 ^^,밝은마음,2010.09.13. 10:34,안녕하세요\n다발성경화증에 관란 정보를 검색하다가 가입하게 되었습니다.\n이런 카페...,"[안녕하세요 반갑습니다 밝은마음님! 카페에서 자주 봬요 ^^, ☎비‥아‥그‥라 ⇒ ..."


In [26]:
# contents_title, contents_text 선택하여 합친 후 1개의 시리즈로 만들어서 list로 변환
gowithms_series = df_gowithms['title'] + ' ' + df_gowithms['contents']
gowithms_series.isna().sum()

0

In [27]:
gowithms_list = gowithms_series.tolist()
gowithms_list

['소아 시신경 염증 (다발성경화증 유전 가능성) 안녕하세요.\n조카의 엄마가 다발성경화증을 앓았고.. (증상: 안구가 중심에서 바깥으로 돌아감)\n가정 내 사정으로, 조카를 다른 가족들이 돌보고 있습니다. 조카는 이제 10세구요.\n최근에 안과를 가보니 눈에 염증이 심하게 껴있었고..\n안과 선생님께서 6개월 정도 약을 먹고 다시 상황을 보자 하셨어요.\n그런데 교실에서 칠판이 안보이는 증상이 있고.. 심하게 피곤해 하는데..\n혹시 다발성경화증 유전인가..\n아니면 뱃속에 있을 때, 엄마로부터 스테로이드 부작용 유전을 받은 것일지..\n여러가지 걱정이 되는 상황입니다.\n혹시 다발성경화증이 유전이 되나요? ㅠㅠ\n아래 증상일 때, 어느 병원을 가야할까요?\n- 눈에 염증이 가득 차 있음\n- 먼거리 시력저하\n- 피곤해 함\n- 10세\n- 모계 유전 (다발성경화증)\n조언 부탁드려요.ㅠㅠ',
 '재발한걸까요? 안녕하세요. 20년도에 다발성경화증 진단 받았습니다. 3일 전부터 오른쪽 팔이 근육통? 이 있는것 같더니 2일 전부터는 피부속이 따끔따금 하면서 바늘로 찌르는듯한 느낌이 지속되고 있어요ㅜㅜ 부위도 20cm정도에서 통증이 왔다면 어제부터는 오른쪽 팔이 손등부터 팔뚝있는데까지 있네요.. 처음 다발성경화증 진단 받았을때는 복시가 왔었는데 재발한걸까요?ㅜㅜ',
 '진단을 받기 직전 어떤 증상을 겪으셨나요? 안녕하세요\n29살 여자입니다\n최근 두통과 어지럼증이 심해져 신경과를 다니고 있어요\n신경과에서 처방한 진통제가 잘 안 들어서 mri를 촬영했는데 병변이 보여 다발성경화증 가능성이 있다 하시더라구요.\n아주 어렸을 때(초중등) 자반증이라고 혈관에 염증이 생기는 자가면역질환을 앓았었고\n3년 전에 미니뇌졸중처럼 일시적으로 심한 두통과 눈 앞 흐려짐, 어지러움과 오른쪽 손/다리 힘빠짐과 약간의 마비 증상이 있었어요.\n자반증은 1년 정도 고생하다 어느날 이유없이 좋아졌고\n미니뇌졸중 같은 경험은 하루 지나니 괜찮아졌습니다\n최근 두통이 심해진 이후로 두개내

###### guillainbarre

In [28]:
df_guillainbarre = dbconnect("persona1_navercafe_guillainbarre")
df_guillainbarre

,_id,cafe,title,name,date,contents,review
0,662b27d42f4a7252bb3a3184,"길랑바레, 밀러피셔 증후군 카페",저는 길랑바레증후군 2주차입니다,십만명의하나,2024.04.25. 16:49,저는 너무 몰랐어요\n얼굴이 얼얼하고 마비가오고 아프기시작\n이미 다리근력 소실이 ...,[고생많으셨네요. 천천히 진행되셔서 기다리고 다시 집에 가고 한거같아요.제남편은 급...
1,662b27d82f4a7252bb3a3185,"길랑바레, 밀러피셔 증후군 카페",진행 속도가 빠르긴 한대요. 병원에서 치료를 위한 속도가 너무 빠른지 문의드려요 (...,mmmj,2024.04.25. 00:20,최초 의심되는 상황\n3-4 일 전 감기 (기침 코)\n주사 맞고 약 처방 받아서 ...,[기관절개 2주까지 버티다가 하세요. 왜 3일만에 하라고 하는지 이해불가네요. 저희...
2,662b27dd2f4a7252bb3a3186,"길랑바레, 밀러피셔 증후군 카페",20대 후반 길랑바레 증후군 재활 중 입니다. 백신 맞지 마세요.,FUCKUVACCINE,2024.04.24. 23:02,감기나 병원도 잘 가지 않던 건강했던 몸이 3년전 코로나 백신을 맞고 달라졌습니다....,"[백신접종 후 얼마만에 증상이 나타났는지요?, 젊으시면 회복되서 일상생활 가능하실거..."
3,662b27e12f4a7252bb3a3187,"길랑바레, 밀러피셔 증후군 카페",복시로만도 밀러피셔라고 할수 있나요??,김또롱또롱,2024.04.23. 23:39,복시증상이 발생해서\n대학병원가서 MRI MRA 척수검사 진행했는데\n이상없다고 하...,[저는 얼굴마비 안구마비 복시살짝 왔었는데 mri 척수검사 피검 다 안나왔는데 신경...
4,662b27e52f4a7252bb3a3188,"길랑바레, 밀러피셔 증후군 카페",길랑바레 판결,냉기도사,2024.04.22. 18:28,"[단독] 독감백신 맞고 희소 신경질환 발병... 법원 ""인과관계 인정""\n인플루엔자...",
...,...,...,...,...,...,...,...
260,66416433e39293e99d9abc50,"길랑바레 증후군(GBS), 밀러피셔 증후군(MFS) 카페",길랑바레 증후군의 예후,spanishsup,2008.04.08. 17:36,1) 전체적으로 사망률은 10% 정도이다.\n\n2) 호흡부전(respiratory...,[병원부터가셔야죠 조금만 불편하신건가요?]
261,6641643ae39293e99d9abc51,"길랑바레 증후군(GBS), 밀러피셔 증후군(MFS) 카페",길랑바레 증후군의 치료법,spanishsup,2008.04.08. 17:35,1) 혈장교환(plasmapheresis)\n이는 자가면역성 질환이라는 기전의 측면...,[병원부터가셔야죠 조금만 불편하신건가요?]
262,66416440e39293e99d9abc52,"길랑바레 증후군(GBS), 밀러피셔 증후군(MFS) 카페",길랑바레 증후군 진단방법,spanishsup,2008.04.08. 17:33,- GBS는 급성 면역성 장애 (acute immunemediated disorde...,[병원부터가셔야죠 조금만 불편하신건가요?]
263,66416446e39293e99d9abc53,"길랑바레 증후군(GBS), 밀러피셔 증후군(MFS) 카페",길랑바레 증후군 임상적 증상,spanishsup,2008.04.08. 17:31,"* 임상적 양상 *\n\n말초신경, 척수후근(Spinal dorsal root), ...",[병원부터가셔야죠 조금만 불편하신건가요?]


In [29]:
# 중복값 제거
df_guillainbarre = drop_duplicates(df_guillainbarre, 'contents')

In [30]:
# 각 컬럼별 전처리(제목)
df_guillainbarre['title']

0                                      저는 길랑바레증후군 2주차입니다
1      진행 속도가 빠르긴 한대요. 병원에서 치료를 위한 속도가 너무 빠른지 문의드려요 (...
2                   20대 후반 길랑바레 증후군 재활 중 입니다. 백신 맞지 마세요.
3                                  복시로만도 밀러피셔라고 할수 있나요??
4                                                길랑바레 판결
                             ...                        
260                                         길랑바레 증후군의 예후
261                                        길랑바레 증후군의 치료법
262                                        길랑바레 증후군 진단방법
263                                      길랑바레 증후군 임상적 증상
264                                         길랑바레 증후군의 원인
Name: title, Length: 265, dtype: object

In [31]:
# 각 컬럼별 전처리(내용)
df_guillainbarre['contents']

0      저는 너무 몰랐어요\n얼굴이 얼얼하고 마비가오고 아프기시작\n이미 다리근력 소실이 ...
1      최초 의심되는 상황\n3-4 일 전 감기 (기침 코)\n주사 맞고 약 처방 받아서 ...
2      감기나 병원도 잘 가지 않던 건강했던 몸이 3년전 코로나 백신을 맞고 달라졌습니다....
3      복시증상이 발생해서\n대학병원가서 MRI MRA 척수검사 진행했는데\n이상없다고 하...
4      [단독] 독감백신 맞고 희소 신경질환 발병... 법원 "인과관계 인정"\n인플루엔자...
                             ...                        
260    1) 전체적으로 사망률은 10% 정도이다.\n\n2) 호흡부전(respiratory...
261    1) 혈장교환(plasmapheresis)\n이는 자가면역성 질환이라는 기전의 측면...
262    - GBS는 급성 면역성 장애 (acute immunemediated disorde...
263    * 임상적 양상 *\n\n말초신경, 척수후근(Spinal dorsal root), ...
264    1916년 프랑스의 신경과 의사 기앵(George Guillain : 1876∼19...
Name: contents, Length: 265, dtype: object

In [32]:
# 광고글 drop
df_guillainbarre = df_guillainbarre[~df_guillainbarre['contents'].str.contains('레어노트')]

In [33]:
df_guillainbarre.loc[:,'contents'] = df_guillainbarre['contents'].apply(remove_text)
df_guillainbarre

,_id,cafe,title,name,date,contents,review
0,662b27d42f4a7252bb3a3184,"길랑바레, 밀러피셔 증후군 카페",저는 길랑바레증후군 2주차입니다,십만명의하나,2024.04.25. 16:49,저는 너무 몰랐어요\n얼굴이 얼얼하고 마비가오고 아프기시작\n이미 다리근력 소실이 ...,[고생많으셨네요. 천천히 진행되셔서 기다리고 다시 집에 가고 한거같아요.제남편은 급...
1,662b27d82f4a7252bb3a3185,"길랑바레, 밀러피셔 증후군 카페",진행 속도가 빠르긴 한대요. 병원에서 치료를 위한 속도가 너무 빠른지 문의드려요 (...,mmmj,2024.04.25. 00:20,최초 의심되는 상황\n3-4 일 전 감기 (기침 코)\n주사 맞고 약 처방 받아서 ...,[기관절개 2주까지 버티다가 하세요. 왜 3일만에 하라고 하는지 이해불가네요. 저희...
2,662b27dd2f4a7252bb3a3186,"길랑바레, 밀러피셔 증후군 카페",20대 후반 길랑바레 증후군 재활 중 입니다. 백신 맞지 마세요.,FUCKUVACCINE,2024.04.24. 23:02,감기나 병원도 잘 가지 않던 건강했던 몸이 3년전 코로나 백신을 맞고 달라졌습니다....,"[백신접종 후 얼마만에 증상이 나타났는지요?, 젊으시면 회복되서 일상생활 가능하실거..."
3,662b27e12f4a7252bb3a3187,"길랑바레, 밀러피셔 증후군 카페",복시로만도 밀러피셔라고 할수 있나요??,김또롱또롱,2024.04.23. 23:39,복시증상이 발생해서\n대학병원가서 MRI MRA 척수검사 진행했는데\n이상없다고 하...,[저는 얼굴마비 안구마비 복시살짝 왔었는데 mri 척수검사 피검 다 안나왔는데 신경...
4,662b27e52f4a7252bb3a3188,"길랑바레, 밀러피셔 증후군 카페",길랑바레 판결,냉기도사,2024.04.22. 18:28,"[단독] 독감백신 맞고 희소 신경질환 발병... 법원 ""인과관계 인정""\n인플루엔자...",
...,...,...,...,...,...,...,...
260,66416433e39293e99d9abc50,"길랑바레 증후군(GBS), 밀러피셔 증후군(MFS) 카페",길랑바레 증후군의 예후,spanishsup,2008.04.08. 17:36,1) 전체적으로 사망률은 10% 정도이다.\n\n2) 호흡부전(respiratory...,[병원부터가셔야죠 조금만 불편하신건가요?]
261,6641643ae39293e99d9abc51,"길랑바레 증후군(GBS), 밀러피셔 증후군(MFS) 카페",길랑바레 증후군의 치료법,spanishsup,2008.04.08. 17:35,1) 혈장교환(plasmapheresis)\n이는 자가면역성 질환이라는 기전의 측면...,[병원부터가셔야죠 조금만 불편하신건가요?]
262,66416440e39293e99d9abc52,"길랑바레 증후군(GBS), 밀러피셔 증후군(MFS) 카페",길랑바레 증후군 진단방법,spanishsup,2008.04.08. 17:33,- GBS는 급성 면역성 장애 (acute immunemediated disorde...,[병원부터가셔야죠 조금만 불편하신건가요?]
263,66416446e39293e99d9abc53,"길랑바레 증후군(GBS), 밀러피셔 증후군(MFS) 카페",길랑바레 증후군 임상적 증상,spanishsup,2008.04.08. 17:31,"* 임상적 양상 *\n\n말초신경, 척수후근(Spinal dorsal root), ...",[병원부터가셔야죠 조금만 불편하신건가요?]


In [34]:
# title, contents 선택하여 합친 후 1개의 시리즈로 만들어서 list로 변환
guillainbarre_series = df_guillainbarre['title'] + ' ' + df_guillainbarre['contents']
guillainbarre_series.isna().sum()

0

In [35]:
guillainbarre_list = guillainbarre_series.tolist()
guillainbarre_list

['저는 길랑바레증후군 2주차입니다 저는 너무 몰랐어요\n얼굴이 얼얼하고 마비가오고 아프기시작\n이미 다리근력 소실이 오고 있단걸\n미처 모르고 절뚝거리면서\n이겨내려했어요 얼굴이 뻣뻣해지고 그래서인가\n숨이 안 쉬어져서 3일을 못자고 못먹고\n바로 당일진료를 받고 입원했습니다\n의사파업상태가 있어서 병실이 꽉 차서\n응급실에서 대기 척수물빼고 mri검사\n응급실에 입원대기 너무 싫었어요\n의료파업 맨 아픈환자24시간 빛을 못 보는\n저에겐 악몽이였어요 물론 지금 입원한 병원와서\nct혈액 다하고 얼굴이 안 움직인다했는데 몸에\n마비가 어디 한쪽으로 안됐으니 가라하더라구요\n시간이 2주뒤에오라며 집에 가래요\n4월8일에 신경과진료 안면마비 22일에 오라며보냈죠ct찍고 더 심해지는 저는 제가 피카츄인줄알았습니다 전기가 막 흐르듯 찌릿하고이전에 몰랐던 시간의 통증들이 생긴걸 왜방치했는지...그 병원 교수님이 잘보는분이 근육마비는있다고 알려만줬어도...다리 손가락 얼굴신경이다 눌렸거든요\n다시 무작정 외래진료갔더니 놀랬어요\n진행중인데 악화되기직전이라서 사형선고같던\n그순간 무서웠어요 정신이 똑바르니까\n악화된날을 기억해요 4월4일그전 증상은 발에 통증이 땅에 못댈정도로왔어요\n튀어나온 뼈로 그저 염증이니 정형외과가면 되지\n했어요 정말 아팠어요 조금 아픈게 무딘 저인데\n차일피일 미루다가 그것이 전조증상인줄도 모르고\n전조증상 중 하나 설사도 있었어요\n어지럽더니 움직일수없던 4월4일 목도 가누기\n불편하고 아프더니 갑자기 복시가 왔어요\n눈이 돌아가고 괜찮다가도 돌아가고 목이 병신처럼\n늘어져서 옆으로 봐야보이고 눈은 반쯤감아야\n눈이 안 아프고 핸드폰은 본지 3일전부터 볼수있어요\n빛을 못 보니 안대를 병원에서 사용했어요\n다리허벅지 손가락은 아직 힘이 떨어져서 퇴원을해도꾸준히 노력해야할꺼같아요\n얼굴도 아직은 얼얼해요\n평생 찍을 mri는 질리도록 한거 같아요\n아무도 이 확률적은 병에 걸리지 않았으면해요\n면역주사를 하는시간은 초응급상태가 된 그시간\n제발

###### parkinsons

In [36]:
df_parkinsons01 = dbconnect("persona1_navercafe_parkinsons")
df_parkinsons02 = dbconnect("persona1_navercafe_parkinsons_two")

In [37]:
# null값 확인
df_parkinsons01['contents'].dropna()
df_parkinsons02['contents'].dropna()

0       서울대학교 병원은 타병원에서 가져온 유전자 검사 결과는 인정 하지 않고\n다시 서울...
1       어머니께서 내일 예약되있는데 하루에 두가지 다 하는거 몸에 무리일까요 ?\n둘중 하...
2       각 각 다른 병원에서 처방 받았습니다.\n두 군데 약이 비슷한건가요??\n어떤 차이...
3       흡인성폐렴으로 인해 호흡곤란이 와서 서울대병원 응급실에서 기도삽관을 하셔서 현재 중...
4       소뇌위축증을 의심하고 있습니다.\n\n26세 남자 \n\n가족력 없습니다.\n\n제...
                              ...                        
7105    no.109\n정수지\n2006/10/30 00:30:02 IP:59.20.101....
7106    안녕하세요..\n파킨슨병으로 진료중인 김종필 환자입니다..(77년생)\n제가 서울에...
7107    no.112\n윤도리\n2006/11/30 10:02:36 IP:61.78.198....
7108    No.24  파킨슨과 알츠하이머  kwan\n수고 많으십니다. 이렇게 질문하는 사람...
7109    부산백병원, 파킨슨병 무료강좌 개최\n[뉴시스] 2006년 06월 27일(화) 오후...
Name: contents, Length: 7110, dtype: object

In [38]:
# 모든 DataFrame을 열 방향으로 합치기
df_parkinsons = pd.concat([df_parkinsons01, df_parkinsons02], ignore_index=True)

df_parkinsons

,_id,cafe,title,name,date,contents,review
0,6629fa3e048ca607028bceb6,파킨슨병,어머니 모시고 연하검사 받고 왔습니다.,긍정의 힘,2024.04.25. 15:28,"아직 초기시기도 하고 식사는 잘하십니다. 기침, 목이 자주 쉬셔서 이때 미리 병원가...",NaN
1,6629fa42048ca607028bceb7,파킨슨병,"스트레스 종류및 파킨슨병 관계, 해소법100가지",피톤치드,2024.04.25. 14:19,"■ 스트레스의 종류\n스트레스에는 short 스트레스,long 스트레스, 루프 스트...",NaN
2,6629fa45048ca607028bceb8,파킨슨병,뇌파검사를 한다는데 뭐를 검사하고 몇시간 걸릴까요?,힘차고즐겁게,2024.04.25. 13:39,"아빠가 파킨슨 확진받으신지 10년좀 넘으셨고요\n작년 가을,겨울에 5회정도\n기면증...",NaN
3,6629fa48048ca607028bceb9,파킨슨병,어지럼증이 동반되나요??,파킨슨초입,2024.04.25. 08:33,일반적인 파킨슨병도 어지럼증이 동반되나요??\n파킨슨 증후군이 아니여도 어지럼증이 ...,"[어지럼증은 동반되오니 주의하세요, 기립성저혈압에 대해 공부해 보세요.]"
4,6629fa4b048ca607028bceba,파킨슨병,소변 마려워 자꾸 깬다면... ‘병’ 때문일 수도,피톤치드,2024.04.24. 23:54,소변 마려워 자꾸 깬다면... ‘병’ 때문일 수도\n반복적인 야간뇨는 특정 질환의 ...,[스트레스는 만병의 근원이라는 말이 있습니다. 스트레스를 제대로 해소하지 못하고 계...
...,...,...,...,...,...,...,...
28940,663421f1fa3f8ee8f69a3944,파킨슨병,파킨슨병,무스,2006.12.30. 22:44,no.109\n정수지\n2006/10/30 00:30:02 IP:59.20.101....,[]
28941,663421f7fa3f8ee8f69a3945,파킨슨병,고엽제,무스,2006.12.30. 22:43,안녕하세요..\n파킨슨병으로 진료중인 김종필 환자입니다..(77년생)\n제가 서울에...,[]
28942,663421fdfa3f8ee8f69a3946,파킨슨병,소뇌종양과 떨림증,무스,2006.12.30. 22:42,no.112\n윤도리\n2006/11/30 10:02:36 IP:61.78.198....,[]
28943,66342203fa3f8ee8f69a3947,파킨슨병,파킨슨병과 알쯔하이머,무스,2006.12.30. 22:34,No.24 파킨슨과 알츠하이머 kwan\n수고 많으십니다. 이렇게 질문하는 사람...,[]


In [39]:
# 중복값 제거
df_parkinsons = drop_duplicates(df_parkinsons, 'contents')
df_parkinsons

,_id,cafe,title,name,date,contents,review
0,6629fa3e048ca607028bceb6,파킨슨병,어머니 모시고 연하검사 받고 왔습니다.,긍정의 힘,2024.04.25. 15:28,"아직 초기시기도 하고 식사는 잘하십니다. 기침, 목이 자주 쉬셔서 이때 미리 병원가...",NaN
1,6629fa42048ca607028bceb7,파킨슨병,"스트레스 종류및 파킨슨병 관계, 해소법100가지",피톤치드,2024.04.25. 14:19,"■ 스트레스의 종류\n스트레스에는 short 스트레스,long 스트레스, 루프 스트...",NaN
2,6629fa45048ca607028bceb8,파킨슨병,뇌파검사를 한다는데 뭐를 검사하고 몇시간 걸릴까요?,힘차고즐겁게,2024.04.25. 13:39,"아빠가 파킨슨 확진받으신지 10년좀 넘으셨고요\n작년 가을,겨울에 5회정도\n기면증...",NaN
3,6629fa48048ca607028bceb9,파킨슨병,어지럼증이 동반되나요??,파킨슨초입,2024.04.25. 08:33,일반적인 파킨슨병도 어지럼증이 동반되나요??\n파킨슨 증후군이 아니여도 어지럼증이 ...,"[어지럼증은 동반되오니 주의하세요, 기립성저혈압에 대해 공부해 보세요.]"
4,6629fa4b048ca607028bceba,파킨슨병,소변 마려워 자꾸 깬다면... ‘병’ 때문일 수도,피톤치드,2024.04.24. 23:54,소변 마려워 자꾸 깬다면... ‘병’ 때문일 수도\n반복적인 야간뇨는 특정 질환의 ...,[스트레스는 만병의 근원이라는 말이 있습니다. 스트레스를 제대로 해소하지 못하고 계...
...,...,...,...,...,...,...,...
20976,663421f1fa3f8ee8f69a3944,파킨슨병,파킨슨병,무스,2006.12.30. 22:44,no.109\n정수지\n2006/10/30 00:30:02 IP:59.20.101....,[]
20977,663421f7fa3f8ee8f69a3945,파킨슨병,고엽제,무스,2006.12.30. 22:43,안녕하세요..\n파킨슨병으로 진료중인 김종필 환자입니다..(77년생)\n제가 서울에...,[]
20978,663421fdfa3f8ee8f69a3946,파킨슨병,소뇌종양과 떨림증,무스,2006.12.30. 22:42,no.112\n윤도리\n2006/11/30 10:02:36 IP:61.78.198....,[]
20979,66342203fa3f8ee8f69a3947,파킨슨병,파킨슨병과 알쯔하이머,무스,2006.12.30. 22:34,No.24 파킨슨과 알츠하이머 kwan\n수고 많으십니다. 이렇게 질문하는 사람...,[]


In [40]:
df_parkinsons.loc[:,'contents'] = df_parkinsons['contents'].apply(remove_text)
df_parkinsons

,_id,cafe,title,name,date,contents,review
0,6629fa3e048ca607028bceb6,파킨슨병,어머니 모시고 연하검사 받고 왔습니다.,긍정의 힘,2024.04.25. 15:28,"아직 초기시기도 하고 식사는 잘하십니다. 기침, 목이 자주 쉬셔서 이때 미리 병원가...",NaN
1,6629fa42048ca607028bceb7,파킨슨병,"스트레스 종류및 파킨슨병 관계, 해소법100가지",피톤치드,2024.04.25. 14:19,"■ 스트레스의 종류 스트레스에는 short 스트레스,long 스트레스, 루프 스트레...",NaN
2,6629fa45048ca607028bceb8,파킨슨병,뇌파검사를 한다는데 뭐를 검사하고 몇시간 걸릴까요?,힘차고즐겁게,2024.04.25. 13:39,"아빠가 파킨슨 확진받으신지 10년좀 넘으셨고요 작년 가을,겨울에 5회정도 기면증처럼...",NaN
3,6629fa48048ca607028bceb9,파킨슨병,어지럼증이 동반되나요??,파킨슨초입,2024.04.25. 08:33,일반적인 파킨슨병도 어지럼증이 동반되나요?? 파킨슨 증후군이 아니여도 어지럼증이 나...,"[어지럼증은 동반되오니 주의하세요, 기립성저혈압에 대해 공부해 보세요.]"
4,6629fa4b048ca607028bceba,파킨슨병,소변 마려워 자꾸 깬다면... ‘병’ 때문일 수도,피톤치드,2024.04.24. 23:54,소변 마려워 자꾸 깬다면... ‘병’ 때문일 수도 반복적인 야간뇨는 특정 질환의 신...,[스트레스는 만병의 근원이라는 말이 있습니다. 스트레스를 제대로 해소하지 못하고 계...
...,...,...,...,...,...,...,...
20976,663421f1fa3f8ee8f69a3944,파킨슨병,파킨슨병,무스,2006.12.30. 22:44,no.109\n정수지\n2006/10/30 00:30:02 IP:59.20.101....,[]
20977,663421f7fa3f8ee8f69a3945,파킨슨병,고엽제,무스,2006.12.30. 22:43,안녕하세요..\n파킨슨병으로 진료중인 김종필 환자입니다..(77년생)\n제가 서울에...,[]
20978,663421fdfa3f8ee8f69a3946,파킨슨병,소뇌종양과 떨림증,무스,2006.12.30. 22:42,no.112\n윤도리\n2006/11/30 10:02:36 IP:61.78.198....,[]
20979,66342203fa3f8ee8f69a3947,파킨슨병,파킨슨병과 알쯔하이머,무스,2006.12.30. 22:34,No.24 파킨슨과 알츠하이머 kwan\n수고 많으십니다. 이렇게 질문하는 사람...,[]


In [41]:
# title, contents 선택하여 합친 후 1개의 시리즈로 만들어서 list로 변환
parkinsons_series = df_parkinsons['title'] + ' ' + df_parkinsons['contents']
parkinsons_series.isna().sum()
parkinsons_series = parkinsons_series.dropna()

In [42]:
parkinsons_list = parkinsons_series.tolist()
parkinsons_list

['어머니 모시고 연하검사 받고 왔습니다. 아직 초기시기도 하고 식사는 잘하십니다. 기침, 목이 자주 쉬셔서 이때 미리 병원가서 검사 받는게 좋다고 글을 봐서 미리 검사 받고 왔어요. 검사시간은 10분이 안걸리는거 같네요. 다행히 삼킴, 연하곤란 모두 걱정안하셔도 되고 나중에 안좋아지시면 그때 다시 검사 받아보라고 하시네요. 먹는 행복이 굉장히 큰데 안좋으면 어쩌나 걱정했는데 마음이 좀 놓입니다.',
 '스트레스 종류및 파킨슨병 관계, 해소법100가지 ■ 스트레스의 종류 스트레스에는 short 스트레스,long 스트레스, 루프 스트레스로 구분합니다. 1. short 스트레스 말 그대로 짧은 시간 동안 발생하는 스트레스입니다. 예를 들어, 시험,발표, 짧은 다툼 등이 쇼트 스트레스에 속합니다. 쇼트 스트레스는 심박수 증가, 혈압 상승, 호흡 곤란 등의 급격한 신체적 변화를 유발하지만, 스트레스 상황이 해결되면 신체는 빠르게 회복됩니다 2.long 스트레스 말 그대로 길게 장기간 지속되는 스트레스입니다. 쉽게 말하면 상황에 겪는 스트레스입니다. 예를 들어, 과도한 업무, 경제적어려움, 만성 질환,가족 문제, 동료 문제등이롱스트레스에 속합니다. 롱 스트레스는 만성적인 피로, 두통, 소화장애, 불면증 , 우울증 등 다양한 건강 문제를 야기할 수 있습니다  3. 루프 스트레스 스트레스 루프는 스트레스 반응의 과정을 설명하는 모델입니다. 스트레스 상황에 노출되면신체는호르몬분비, 심박수증가, 혈압 상승 등의 변화를 통해 스트레스에 대응합니다. 이러한 변화는 스트레스 해결 후 원상태로 회복됩니다. 하지만 스트레스 상황이 지속되거나 반복적으로 발생하면 스트레스 루프가 계속 회전하게 되고, 이는 만성적인 스트레스로 이어져 건강에 악영향을 미칠 수 있습니다. (일명 무한 루프라고 한다.)  ■ 스트레스와 파킨슨병의 관계? 장기간의 불안과 정서적 충격은 이미 파킨슨병의 주요 원인이라는 얘기가 있을 정도로 오래전부터 언급되었습니다. 자율신경은 인체가 내부 환경 변화나 외부 자극에 대

###### navercafe - 희귀 신경계 질환

In [43]:
df_rare_nervous_disorder01 = dbconnect("naver_cafe_Sympton")
df_rare_nervous_disorder02 = dbconnect("naver_cafe_dongchul")
df_rare_nervous_disorder03 = dbconnect("naver_cafe_yugyeong")
df_rare_nervous_disorder04 = dbconnect("naver_cafe_yujin")
df_rare_nervous_disorder05 = dbconnect("naver_cafe_jisu")

In [44]:
# null값 확인
df_rare_nervous_disorder01.isna().sum()

Series([], dtype: float64)

In [45]:
# 모든 DataFrame을 열 방향으로 합치기
df_rare_nervous_disorder = pd.concat([df_rare_nervous_disorder01, df_rare_nervous_disorder02, df_rare_nervous_disorder03, df_rare_nervous_disorder04, df_rare_nervous_disorder05], ignore_index=True)

df_rare_nervous_disorder

,_id,dise_name,dise_code,cafe,title,name,date,contents,review
0,66417aadc68fb6f104417c71,중증근무력증,G70.0,♥시험관아기 대표카페♥,"혹시, 복시(겹쳐보임)가 있거나 중증근무력증 진단 받으신 분 계실까요?",설탕친토마토,2024.03.20. 08:43,"안녕하세요.\n채취 2회, 동결 5차를 앞두고 있습니다.\n얼마전? 그 이전부터 초...",[신경과에서 소론도정 처방받아서 증상 조절하고 있어요. 저같은 경우는 자연적으로 없...
1,66417ab5c68fb6f104417c73,중증근무력증,G70.0,건강한 리서치 건강/ 미용/ 맛집 /서울 강남구 송파구 강서구 등,중증근무력증 치료 전문병원 시설 좋은곳 어디인지 아세요?(자료수집중),자료정리,2024.05.01. 22:02,반갑습니다 ^^ 카페 여러분들 항상 잘 계시죠?\n이번 리서치는 중증근무력증 치료 ...,[중증근무력증 관련정보 찾는분 은근 계신것 같더라고요 다른 회원님들께 여쭤보시면 될...
2,66417abcc68fb6f104417c75,중증근무력증,G70.0,스마트홀릭 ㅣ스사사맘스홀릭중고나라 여우야 딜공디매쇼핑테무,근력이 저하되는 중증 근무력증 알아 봅니다,미소의마법,2024.04.28. 08:28,근력이 저하되는 중증 근무력증은 근력이 감소하여 근육이 약해지고 기능이 저하되는 질...,[▶테무 50만원 무료쇼핑 할인코드 ▶https://temu.com/s/1d3vDj...
3,66417ac4c68fb6f104417c77,중증근무력증,G70.0,고양이라서 다행이야,중증 근무력증,쌔랄라,2023.07.21. 18:19,"저희 아이가 최근 갑작스럽게 아파 입원중 입니다\nmri,ct 모두 정상이라 나오고...",[바이러스 관련 가능성이 있으니\n\n그 쪽으로 사람 질병으로 검색도 해보세요..\...
4,66417acbc68fb6f104417c79,중증근무력증,G70.0,폐암환우들과 그 가족의 모임,항암중 중증근무력증 치료하신 분 계신가요?,묘성연,2023.08.30. 21:40,아버지께서 비소세포 편평상피 치료 4년차이십니다.\n20년도에 키트루다로 시작해서 ...,[복시가 있나요????전신중증근무력증은 초기에 복시가\n먼저 시작되는 경우가 많습니...
...,...,...,...,...,...,...,...,...,...
13291,6642c0d6205d240ea773906d,모세혈관확장성운동실조,G35,아삭아삭 (원주 과일도소매/과일카페),12/11~12/12 즐거운주말♬♬ 아삭이네 맛있는 과일 올라갑니다.﻿ 달콤한 가정...,아삭아삭,2021.12.10. 23:26,12/11~12/12 즐거운주말♬♬\n아삭이네 맛있는 과일 올라갑니다.\n한가지 안...,
13292,6642c1e14ceea7c26c01e340,모세혈관확장성운동실조,G35,아삭아삭 (원주 과일도소매/과일카페),1/25 화요일♬♬ 아삭이네 맛있는 과일 올라갑니다.,아삭아삭,2022.01.25. 10:47,1/25 화요일♬♬\n아삭이네 맛있는 과일 올라갑니다.\n대량주문건으로 배송이 많이...,
13293,6642c1e84ceea7c26c01e342,모세혈관확장성운동실조,G35,♬~수원맘 모여라,광교변비 탈출하기~ 장편한외과와 알아보는 반복되는 변비~!!,장편한외과,2022.02.21. 09:34,광교 변비 장편한외과\n새해를 지나면서 건강하고 아름다워지기 위해 헬스장 등에서 구...,[예약 하고가야되나요? 대장내시경 받은지 얼마 안됬는데 혈변을 보고 변비 설사 반복...
13294,6642c1f14ceea7c26c01e344,모세혈관확장성운동실조,G35,"내보험속삭임 - 건강,암,태아,종신,치매,간병인,사망,수술비보험",MG손해보험 ‘원더풀 종합보험’ 보험 하나로 100세까지 완벽 보장 받아보세요,모드니,2020.08.22. 23:05,MG손해보험 ‘원더풀 종합보험’\n보험 하나로 100세까지 완벽 보장 받아보세요\n...,


In [46]:
# 중복값 제거
df_rare_nervous_disorder = drop_duplicates(df_rare_nervous_disorder, 'contents')
df_rare_nervous_disorder

,_id,dise_name,dise_code,cafe,title,name,date,contents,review
0,66417aadc68fb6f104417c71,중증근무력증,G70.0,♥시험관아기 대표카페♥,"혹시, 복시(겹쳐보임)가 있거나 중증근무력증 진단 받으신 분 계실까요?",설탕친토마토,2024.03.20. 08:43,"안녕하세요.\n채취 2회, 동결 5차를 앞두고 있습니다.\n얼마전? 그 이전부터 초...",[신경과에서 소론도정 처방받아서 증상 조절하고 있어요. 저같은 경우는 자연적으로 없...
1,66417ab5c68fb6f104417c73,중증근무력증,G70.0,건강한 리서치 건강/ 미용/ 맛집 /서울 강남구 송파구 강서구 등,중증근무력증 치료 전문병원 시설 좋은곳 어디인지 아세요?(자료수집중),자료정리,2024.05.01. 22:02,반갑습니다 ^^ 카페 여러분들 항상 잘 계시죠?\n이번 리서치는 중증근무력증 치료 ...,[중증근무력증 관련정보 찾는분 은근 계신것 같더라고요 다른 회원님들께 여쭤보시면 될...
2,66417abcc68fb6f104417c75,중증근무력증,G70.0,스마트홀릭 ㅣ스사사맘스홀릭중고나라 여우야 딜공디매쇼핑테무,근력이 저하되는 중증 근무력증 알아 봅니다,미소의마법,2024.04.28. 08:28,근력이 저하되는 중증 근무력증은 근력이 감소하여 근육이 약해지고 기능이 저하되는 질...,[▶테무 50만원 무료쇼핑 할인코드 ▶https://temu.com/s/1d3vDj...
3,66417ac4c68fb6f104417c77,중증근무력증,G70.0,고양이라서 다행이야,중증 근무력증,쌔랄라,2023.07.21. 18:19,"저희 아이가 최근 갑작스럽게 아파 입원중 입니다\nmri,ct 모두 정상이라 나오고...",[바이러스 관련 가능성이 있으니\n\n그 쪽으로 사람 질병으로 검색도 해보세요..\...
4,66417acbc68fb6f104417c79,중증근무력증,G70.0,폐암환우들과 그 가족의 모임,항암중 중증근무력증 치료하신 분 계신가요?,묘성연,2023.08.30. 21:40,아버지께서 비소세포 편평상피 치료 4년차이십니다.\n20년도에 키트루다로 시작해서 ...,[복시가 있나요????전신중증근무력증은 초기에 복시가\n먼저 시작되는 경우가 많습니...
...,...,...,...,...,...,...,...,...,...
10750,6642ae93986e1adbfba2fe2a,모세혈관확장성운동실조,G11.3,통합치유연구소[통치연] 심혈관/암/당뇨/자가면역/정신질환/,스테로이드 호르몬 크림 장기 사용과 피부 문제,연구소장,2019.01.19. 14:50,"스테로이드 호르몬 크림 장기 사용과 피부 문제\n\n\n습진, 건선, 아토피 피부염...",
10751,6642ae9b986e1adbfba2fe2b,모세혈관확장성운동실조,G11.3,암(만성병) 극복의 길을 찾는 사람들,"[해외-영어책] Nutritional Oncology 영양 종양학 - 암 예방, 치...",꿈많은사람2,2021.12.27. 21:52,"--- 작성자썰 ----\n*. 역시나, 바쁘신분들은 그냥 패스해주세요. 그냥 혼자...","[정성에 감사드립니다., 🕵️😁탐구 정신, 한글 번역판이 이번에 출판된 것으로 알고..."
10752,6642aea2986e1adbfba2fe2c,모세혈관확장성운동실조,G11.3,황정의 자연치유,어쩌면 혈이 전부다 2 (고혈압과 저혈압은.. ),카렌,2020.08.11. 18:52,서양의학이 아닌 동양 의학에서는\n건강 장수 할 수 있는 세가지 요건은\n결국 기가...,"[의사 리포트 읽는 것 같아요. ㅋ 그런데 귀에 쏙쏙 들어오네요. ㅋ, 넘 길지요?..."
10753,6642aeb1986e1adbfba2fe2e,모세혈관확장성운동실조,G11.3,똑똑한투병기,종양 표지자(tumor marker) 검사란 무엇인가,제3의 인생,2020.07.21. 13:13,1. 종양표지자의 이용\n\n\n1) 암의 선별과 조기 진단 (Screening)\...,"[감사합니다, 감사합니다\n잘읽었습니다, 좋은 자료 감사합니다♡]"


In [47]:
# title, contents 선택하여 합친 후 1개의 시리즈로 만들어서 list로 변환
rare_nervous_disorder_series = df_rare_nervous_disorder['title'] + ' ' + df_rare_nervous_disorder['contents']
rare_nervous_disorder_series.isna().sum()
rare_nervous_disorder_series = rare_nervous_disorder_series.dropna()

In [48]:
rare_nervous_disorder_list = rare_nervous_disorder_series.tolist()

###### navercafe - 희귀질환 외 신경계 질환

In [49]:
df_nervous_disorder01 = dbconnect("naver_cafe_yujin_G00-G09")
df_nervous_disorder02 = dbconnect("naver_cafe_G00-G09")
df_nervous_disorder03 = dbconnect("naver_cafe_G10-G14")
df_nervous_disorder04 = dbconnect("naver_cafe_G20-G26")
df_nervous_disorder05 = dbconnect("naver_cafe_G40-G47")
df_nervous_disorder06 = dbconnect("naver_cafe_G80-G83")

In [50]:
# 두 DataFrame을 열 방향으로 합치기
df_nervous_disorder = pd.concat([df_nervous_disorder01, df_nervous_disorder02, df_nervous_disorder03, df_nervous_disorder04, df_nervous_disorder05, df_nervous_disorder06], ignore_index=True)

df_nervous_disorder

,_id,dise_name,dise_code,cafe,title,name,date,contents,review
0,6641dbe11637bfeab2b24930,세균성 수막염,G00.9,느린걸음(장애 보호자 및 재활 전문가 모임 ),바이러스성 / 세균성 뇌수막염,달맘,2022.01.11. 22:10,"친구 아기가 지금 생후 70일 조금 넘었고, 2주전쯤 예방접종 맞은 후로 아기 컨디...",[그러니까 말이에요 ㅜㅜ 그래서 일단 정확히 다시 물어보라구 했어요. 만에 하나 세...
1,6641dbe91637bfeab2b24931,세균성 수막염,G00.9,느린걸음(장애 보호자 및 재활 전문가 모임 ),37일 아기 세균성 뇌수막염,서율서아맘,2023.09.22. 11:14,생후 37일에 38.5도 고열로 인해 응급실 내원하였고 각종 검사 진행하였는데\n-...,[안녕하세요. 저희는 생후 1일차에 숨이 가빠져서 대학병원 옮기는 도중에 열이 오르...
2,6641dbf11637bfeab2b24932,세균성 수막염,G00.9,"맘스홀릭 베이비 (330만 엄마들의 임신, 육아 지식in 카페)",이유없는 고열 병원가니깐 뇌수막염 세균성 / 가와사키라는데 치료받으신분 계신가요 후...,뿌규라쀼,2024.04.15. 02:45,제목 그대로 이유없는 고열로 병원갔더니\n염증수치 8.6대로 정상범위보다 높긴 했지...,"[가와사키 작년 12월 진단후 완치 판정 받았어요! 휴우증은 없어요, ㅠㅠ 다행이네..."
3,6641dbf81637bfeab2b24933,세균성 수막염,G00.9,"맘스홀릭 베이비 (330만 엄마들의 임신, 육아 지식in 카페)",4살 세균성뇌수막염,rkdgp03,2023.11.27. 21:23,단순히 감기겠지싶어서 5일동안 감기먹여도 열이 안떨어지고 머리가 너무아프다는 말에 ...,"[톼원 하셨나요?, 저희 아기도 세균성뇌수막염으로 입원중이에요.. 앞으로 3주는 더..."
4,6641dbff1637bfeab2b24934,세균성 수막염,G00.9,"곰 마을 재테크 (월급쟁이,펀드,보험,주식,예적금,코인,부동산,)",태아보험 중증세균성수막염진단담보 큰 위험에 대비하세요~,다온맘s,2020.07.02. 13:51,<br />\n\n\n안녕하세요~\n보험판다 스탭다온맘s입니다~\n\n벌써 7월이네...,"[중증수막염 정말 위험하죠. 꼭 준비하셨음 좋겠어요 ~, 네!!맞아요..꼭 준비하셔..."
...,...,...,...,...,...,...,...,...,...
7497,66431871ccdb9249a0899ca8,강직성 뇌성마비,G80.0,광남교회사랑해,건강상식,박기복,2011.02.04. 18:55,<FORM name=formBGM2><INPUT onchange=setVolume2...,[☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆☆\n\n정관수술방법 관...
7498,6643187eccdb9249a0899ca9,강직성 뇌성마비,G80.0,"물작메, 물리치료에 관한 작은 메모장, 물리치료, 물리치료사",생화학 용어 정리,onlyjaejin,2008.09.21. 23:50,"L chain (경쇄, 경사슬, L쇄)\nL-amino acid oxidase ...","[감사합니다.~, 감사합니다., biochemistry]"
7499,6643188accdb9249a0899caa,강직성 뇌성마비,G80.0,옹달샘 ♡ 중증아이care,안녕하세요 :),룩북86설이20,2022.10.23. 20:11,1. 닉네임변경 변경하셨나요?\n옹달샘(엄마출생년도)아이이름(아이출생년도) 아이이름...,"[안녕하세요 이곳에서 정보와 힐링 얻으셔요 반갑습니다, 네 감사합니다 :) 많은 정..."
7500,66431892ccdb9249a0899cab,강직성 뇌성마비,G80.0,코어발란스핏,무정위 운동성(Athetosis type) 뇌성마비 아동의 치료,코어마스터 OSH,2011.08.11. 10:09,1. 시작하는 말\n\n이 글은 근육 긴장도(muscle tone)의 기복이 있는\...,


In [51]:
# 중복값 제거
df_nervous_disorder = drop_duplicates(df_nervous_disorder, 'contents')
df_nervous_disorder

,_id,dise_name,dise_code,cafe,title,name,date,contents,review
0,6641dbe11637bfeab2b24930,세균성 수막염,G00.9,느린걸음(장애 보호자 및 재활 전문가 모임 ),바이러스성 / 세균성 뇌수막염,달맘,2022.01.11. 22:10,"친구 아기가 지금 생후 70일 조금 넘었고, 2주전쯤 예방접종 맞은 후로 아기 컨디...",[그러니까 말이에요 ㅜㅜ 그래서 일단 정확히 다시 물어보라구 했어요. 만에 하나 세...
1,6641dbe91637bfeab2b24931,세균성 수막염,G00.9,느린걸음(장애 보호자 및 재활 전문가 모임 ),37일 아기 세균성 뇌수막염,서율서아맘,2023.09.22. 11:14,생후 37일에 38.5도 고열로 인해 응급실 내원하였고 각종 검사 진행하였는데\n-...,[안녕하세요. 저희는 생후 1일차에 숨이 가빠져서 대학병원 옮기는 도중에 열이 오르...
2,6641dbf11637bfeab2b24932,세균성 수막염,G00.9,"맘스홀릭 베이비 (330만 엄마들의 임신, 육아 지식in 카페)",이유없는 고열 병원가니깐 뇌수막염 세균성 / 가와사키라는데 치료받으신분 계신가요 후...,뿌규라쀼,2024.04.15. 02:45,제목 그대로 이유없는 고열로 병원갔더니\n염증수치 8.6대로 정상범위보다 높긴 했지...,"[가와사키 작년 12월 진단후 완치 판정 받았어요! 휴우증은 없어요, ㅠㅠ 다행이네..."
3,6641dbf81637bfeab2b24933,세균성 수막염,G00.9,"맘스홀릭 베이비 (330만 엄마들의 임신, 육아 지식in 카페)",4살 세균성뇌수막염,rkdgp03,2023.11.27. 21:23,단순히 감기겠지싶어서 5일동안 감기먹여도 열이 안떨어지고 머리가 너무아프다는 말에 ...,"[톼원 하셨나요?, 저희 아기도 세균성뇌수막염으로 입원중이에요.. 앞으로 3주는 더..."
4,6641dbff1637bfeab2b24934,세균성 수막염,G00.9,"곰 마을 재테크 (월급쟁이,펀드,보험,주식,예적금,코인,부동산,)",태아보험 중증세균성수막염진단담보 큰 위험에 대비하세요~,다온맘s,2020.07.02. 13:51,<br />\n\n\n안녕하세요~\n보험판다 스탭다온맘s입니다~\n\n벌써 7월이네...,"[중증수막염 정말 위험하죠. 꼭 준비하셨음 좋겠어요 ~, 네!!맞아요..꼭 준비하셔..."
...,...,...,...,...,...,...,...,...,...
3878,66431807ccdb9249a0899c9d,강직성 뇌성마비,G80.0,"물작메, 물리치료에 관한 작은 메모장, 물리치료, 물리치료사",재활의학 필수용어,마미카르,2009.04.25. 22:57,"- 아래 주의사항은 삭제하지 마시고, 게시글은 최하단에 올려주세요 -\n------...","[자료감사합니다. ^^, 감사해여 ^^, 감사합니다~~, 많군요 ㅋㅋ, 잘봣어요, ..."
3879,66431838ccdb9249a0899ca2,강직성 뇌성마비,G80.0,앉다서다걷다,경직성 양하지마비 아동과 엉덩이관절 탈구 - 치료 훈련중인 아동의 정형외과적 수술 ...,미소담은,2020.11.22. 05:40,미소담은재활운동연구소 박원장입니다.\n이번 글에서는 현재 제가 거의 5년간 소아재활...,[이해하기 어려운 부분이 있지만 전문적인 내용이라 미처 몰랐던 많은 것을 알게 되었...
3880,6643184cccdb9249a0899ca4,강직성 뇌성마비,G80.0,darkeyes,뇌성마비,무공저,2014.06.21. 23:46,"뇌성마비 (한자: 腦性痲痺; 영어: cerebral palsy, CP)는 중추신경계...",
3881,66431854ccdb9249a0899ca5,강직성 뇌성마비,G80.0,앉다서다걷다,"재활훈련의 강도, 시간 그리고 휴식을 잘 고려하세요.",미소담은,2022.03.09. 02:27,그림1 몸통 근력 훈련\n그림2 몸통 근력 훈련\n그림3 보행 훈련\n소아재활훈련 ...,


In [52]:
df_nervous_disorder.loc[:,'contents'] = df_nervous_disorder['contents'].apply(remove_text)
df_nervous_disorder

,_id,dise_name,dise_code,cafe,title,name,date,contents,review
0,6641dbe11637bfeab2b24930,세균성 수막염,G00.9,느린걸음(장애 보호자 및 재활 전문가 모임 ),바이러스성 / 세균성 뇌수막염,달맘,2022.01.11. 22:10,"친구 아기가 지금 생후 70일 조금 넘었고, 2주전쯤 예방접종 맞은 후로 아기 컨디...",[그러니까 말이에요 ㅜㅜ 그래서 일단 정확히 다시 물어보라구 했어요. 만에 하나 세...
1,6641dbe91637bfeab2b24931,세균성 수막염,G00.9,느린걸음(장애 보호자 및 재활 전문가 모임 ),37일 아기 세균성 뇌수막염,서율서아맘,2023.09.22. 11:14,생후 37일에 38.5도 고열로 인해 응급실 내원하였고 각종 검사 진행하였는데\n-...,[안녕하세요. 저희는 생후 1일차에 숨이 가빠져서 대학병원 옮기는 도중에 열이 오르...
2,6641dbf11637bfeab2b24932,세균성 수막염,G00.9,"맘스홀릭 베이비 (330만 엄마들의 임신, 육아 지식in 카페)",이유없는 고열 병원가니깐 뇌수막염 세균성 / 가와사키라는데 치료받으신분 계신가요 후...,뿌규라쀼,2024.04.15. 02:45,제목 그대로 이유없는 고열로 병원갔더니\n염증수치 8.6대로 정상범위보다 높긴 했지...,"[가와사키 작년 12월 진단후 완치 판정 받았어요! 휴우증은 없어요, ㅠㅠ 다행이네..."
3,6641dbf81637bfeab2b24933,세균성 수막염,G00.9,"맘스홀릭 베이비 (330만 엄마들의 임신, 육아 지식in 카페)",4살 세균성뇌수막염,rkdgp03,2023.11.27. 21:23,단순히 감기겠지싶어서 5일동안 감기먹여도 열이 안떨어지고 머리가 너무아프다는 말에 ...,"[톼원 하셨나요?, 저희 아기도 세균성뇌수막염으로 입원중이에요.. 앞으로 3주는 더..."
4,6641dbff1637bfeab2b24934,세균성 수막염,G00.9,"곰 마을 재테크 (월급쟁이,펀드,보험,주식,예적금,코인,부동산,)",태아보험 중증세균성수막염진단담보 큰 위험에 대비하세요~,다온맘s,2020.07.02. 13:51,<br />\n\n\n안녕하세요~\n보험판다 스탭다온맘s입니다~\n\n벌써 7월이네...,"[중증수막염 정말 위험하죠. 꼭 준비하셨음 좋겠어요 ~, 네!!맞아요..꼭 준비하셔..."
...,...,...,...,...,...,...,...,...,...
3878,66431807ccdb9249a0899c9d,강직성 뇌성마비,G80.0,"물작메, 물리치료에 관한 작은 메모장, 물리치료, 물리치료사",재활의학 필수용어,마미카르,2009.04.25. 22:57,"- 아래 주의사항은 삭제하지 마시고, 게시글은 최하단에 올려주세요 -\n------...","[자료감사합니다. ^^, 감사해여 ^^, 감사합니다~~, 많군요 ㅋㅋ, 잘봣어요, ..."
3879,66431838ccdb9249a0899ca2,강직성 뇌성마비,G80.0,앉다서다걷다,경직성 양하지마비 아동과 엉덩이관절 탈구 - 치료 훈련중인 아동의 정형외과적 수술 ...,미소담은,2020.11.22. 05:40,미소담은재활운동연구소 박원장입니다.\n이번 글에서는 현재 제가 거의 5년간 소아재활...,[이해하기 어려운 부분이 있지만 전문적인 내용이라 미처 몰랐던 많은 것을 알게 되었...
3880,6643184cccdb9249a0899ca4,강직성 뇌성마비,G80.0,darkeyes,뇌성마비,무공저,2014.06.21. 23:46,"뇌성마비 (한자: 腦性痲痺; 영어: cerebral palsy, CP)는 중추신경계...",
3881,66431854ccdb9249a0899ca5,강직성 뇌성마비,G80.0,앉다서다걷다,"재활훈련의 강도, 시간 그리고 휴식을 잘 고려하세요.",미소담은,2022.03.09. 02:27,그림1 몸통 근력 훈련\n그림2 몸통 근력 훈련\n그림3 보행 훈련\n소아재활훈련 ...,


In [53]:
# title, contents 선택하여 합친 후 1개의 시리즈로 만들어서 list로 변환
nervous_disorder_series = df_nervous_disorder['title'] + ' ' + df_nervous_disorder['contents']
nervous_disorder_series.isna().sum()
nervous_disorder_series.dropna()

0       바이러스성 / 세균성 뇌수막염 친구 아기가 지금 생후 70일 조금 넘었고, 2주전쯤...
1       37일 아기 세균성 뇌수막염 생후 37일에 38.5도 고열로 인해 응급실 내원하였고...
2       이유없는 고열 병원가니깐 뇌수막염 세균성 / 가와사키라는데 치료받으신분 계신가요 후...
3       4살 세균성뇌수막염 단순히 감기겠지싶어서 5일동안 감기먹여도 열이 안떨어지고 머리가...
4       태아보험 중증세균성수막염진단담보 큰 위험에 대비하세요~ <br />\n\n\n안녕하...
                              ...                        
3878    재활의학 필수용어 - 아래 주의사항은 삭제하지 마시고, 게시글은 최하단에 올려주세요...
3879    경직성 양하지마비 아동과 엉덩이관절 탈구 - 치료 훈련중인 아동의 정형외과적 수술 ...
3880    뇌성마비 뇌성마비 (한자: 腦性痲痺; 영어: cerebral palsy, CP)는 ...
3881    재활훈련의 강도, 시간 그리고 휴식을 잘 고려하세요. 그림1 몸통 근력 훈련\n그림...
3882    신경해부 Anatomy an Vascular Territories of the Th...
Length: 3883, dtype: object

In [54]:
nervous_disorder_list = nervous_disorder_series.tolist()

##### 형태소 분석

In [55]:
# 모든 자연어 list 합치기
total_lists = KMSS_list + gowithms_list + guillainbarre_list + rare_nervous_disorder_list

###### 불용어 제거

In [56]:
import re
patterns = '()'
series = []
for total_list in total_lists:
    total_list = str(total_list)
    text_regex = re.sub(pattern=patterns,repl='',string=total_list)
    if text_regex != '':
        series.append(text_regex)
series[:2]

["늦었습니다. 어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.오늘 아침에 퇴원을 하셔서 집에 와서 여러가지 일을 하다보니 글 올리는 것이 생각이 났습니다.;오늘은 우리가 많이 먹는 포도에 대한 효능과 부작용에 대한 글입니다.포도 효능 11가지한여름이 지나가고 가을이 올 때쯤 되면 한참 달콤한 과일을 먹을 수 있어서 좋은 거 같다. 달콤한 복숭아도 좋고, 새콤한 자두도 좋지만 한 알 한 알 톡톡 터지는 식감이 좋은 포도도 너무 좋아해요. 아이들 간식부터 어르신들 입맛까지 사로잡은 과일로 누구나 좋아하는 음식으로 손꼽힌다.7~8월이 제철이어서 무더운 여름에 맛있게 먹을 수 있는 과일입니다. 특유의 향이 좋고 과즙이 풍부해서 과일 자체로도 맛있지만 주스로 마시거나 건포도로 가공해서 먹기도 한다. 우리나라는 이 과일을 꽤 옛날부터 먹어왔습니다.포도 효능1. 혈압을 조절해 준다.포도는 심장 건강에 도움이 되는 한 부분으로 혈압에 긍정적인 영향을 미칩니다. 혈압은 동맥혈관 벽에 대항한 혈액의 압력을 말한다. 이 혈액의 압력이 여러 원인으로 인해 높아진 상태를 고혈압이라 하는데, 우리나라 성인 인구의 30%가량이 고혈압 증상이 있는 것으로 추정하고 있습니다. 포도에는 칼륨과 섬유질이 풍부해서 나트륨의 배출을 돕습니다. 나트륨이 고혈압을 유발하는 원인 중 한 가지입니다. 포도를 적정량 꾸준히 섭취하다 보면 나트륨으로 인한 고혈압의 위험을 낮출 수 있습니다.2. 치매를 예방해 준다.포도 씨의 폴리페놀 성분은 알츠하이머의 진전을 늦춰줍니다. 연구에 의하면 실험용 생쥐에게 포도 씨의 폴리페놀 추출물을 5개월 동안 투여한 결과, 기억력 감퇴에 영향을 주는 ‘아밀로이드 베타-56’이라 하는 신경독소 물질이 감소한 것으로 나타났습니다.3. 항암 효과를 볼 수 있다.포도에는 폴리페놀이라는 항산화 성분이 포함되어 있습니다. 이들은 산화방지제 역할을 하여 암 관리와과 세포 손상 및 보호에 도움을 줄 수 있습니다. 특히 적포도에는 레스베라트롤이라는 폴리페놀의 일종인 성분이 많이 함유

In [91]:
# 불용어 리스트 생성
f=open('./total_raredisease_community_stopwords.txt')
stopwords=[]
lines = f.readlines()
for line in lines:
    line = line.strip()
    stopwords.append(line)
f.close()

###### Tokenizing

In [92]:
from konlpy.tag import Okt

# Okt 형태소 분석기 인스턴스 생성
okt = Okt()

# 토크나이징 함수 정의
def tokenizer(raw, pos=["Noun", "Adverb"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 그랰ㅋㅏ -> 그래ㅋㅋ
            stem=True    # stemming 바뀌나->바뀌다
            )
            if len(word) > 1 and tag in pos and word not in stopword
    ]

In [93]:
# tokenizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
# tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.95, min_df=2)
tfidfVectorizer = TfidfVectorizer(tokenizer=tokenizer, max_df=0.80, min_df=3)

In [94]:
series = [' '.join(x.split()) for x in series]
features = tfidfVectorizer.fit_transform(series)
features.toarray()[:2]

/usr/local/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

###### LDA topic 추출

LDA 시각화

In [95]:
import pyLDAvis
import pyLDAvis.lda_model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation

In [96]:
lda_model  = LatentDirichletAllocation(n_components=3, n_jobs=2) # n_components => topic 가짓수 결정 / n_jobs =>성능높이기
lda_model.fit(features)

LatentDirichletAllocation(n_components=3, n_jobs=2)

In [97]:
vis = pyLDAvis.lda_model.prepare(lda_model, features, tfidfVectorizer) # 토픽모델, 교육이 끝난 값(행렬형태), 교육모델

In [98]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis) # PCA - 차원축소

In [99]:
total_lists

filter = []
for total_list in total_lists:
    try:
        if '피린' in total_list:
            total_list = str(total_list)
            print(total_list)
            print("--------")
        else : 
            pass
    except:
        print(total_list)
        pass

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



LDA topic 군집분석

In [100]:
dictionary_list = tfidfVectorizer.get_feature_names_out(features)
dictionary_list

array(['가가', '가가린', '가가호호', ..., '힙싯', '힙업', '힙합'], dtype=object)

In [101]:
lda_model.components_

array([[4.371057  , 0.34895125, 0.43611023, ..., 0.33462366, 0.44699476,
        0.43700162],
       [1.32066992, 0.47062583, 0.35088844, ..., 0.33952846, 0.35412005,
        1.45775476],
       [0.3361627 , 0.33413577, 0.33418232, ..., 0.33336928, 0.33480403,
        0.33442227]])

In [102]:
topics_output = lda_model.transform(features)
df_topics_score = pd.DataFrame(data=topics_output)

In [103]:
df_topics_score['dominant_topic_number'] = np.argmax(topics_output, axis=1)

In [104]:
df_topics_score['sentences'] = series
df_topics_score

,0,1,2,dominant_topic_number,sentences
0,0.939293,0.031155,0.029552,0,늦었습니다. 어머님 병간호하늘가고 병원에 가서 2주일을 보냈습니다.오늘 아침에 퇴원...
1,0.317759,0.617948,0.064293,1,Only Love - Nana Mouskouri: with Lyrics 상대에게 신...
2,0.170961,0.740911,0.088128,1,하비(Harvey) https://youtu.be/q_TJlMUHfp4하비(Harv...
3,0.389516,0.548744,0.061740,1,4월아 안녕~~. 4월이 마침표를 찍는 날입니다.날씨가 변화무쌍했습니다.(추웠다 더...
4,0.237068,0.668111,0.094822,1,4월을 마무리 짓는 날입니다. 오늘로서 4월이라는 달을 마무리 짓고 내일부터는 가정...
...,...,...,...,...,...
12309,0.939803,0.030136,0.030061,0,스테로이드 호르몬 크림 장기 사용과 피부 문제 스테로이드 호르몬 크림 장기 사용과 ...
12310,0.753509,0.216069,0.030422,0,"[해외-영어책] Nutritional Oncology 영양 종양학 - 암 예방, 치..."
12311,0.940218,0.030867,0.028916,0,어쩌면 혈이 전부다 2 (고혈압과 저혈압은.. ) 서양의학이 아닌 동양 의학에서는 ...
12312,0.951113,0.024433,0.024454,0,종양 표지자(tumor marker) 검사란 무엇인가 1. 종양표지자의 이용 1) ...


In [105]:
## 상위 단어 추출
## 0 확률 1은 dictionary
topics_list = list()
for topic in lda_model.components_:
    df_datas = [topic, dictionary_list]
    df_topics = pd.DataFrame(data=df_datas)
    df_topics= df_topics.T
    df_topics = df_topics.sort_values(0, ascending=False)
    # print(df_topics[:3])
    topics_text = ' '.join(df_topics[1].values[:10])# 시리즈 형식으로 출력 get values from series / index
    print(topics_text)
    topics_list.append(topics_text)


topics_list_add = [topics_list, ['Topic0', 'Topic1', 'Topic2']]
df_topics_keywords = pd.DataFrame(topics_list_add)

뇌전증 장애 유전자 근육 발작 원인 세포 경련 시신경 입원
그녀 당신 대통령 케네디 세계 남자 인간 전쟁 소설 무슨
호전 한의원 한약 택일 양방 약간 말기 현실 초기 필요


In [106]:
df_topics_keywords

,0,1,2
0,뇌전증 장애 유전자 근육 발작 원인 세포 경련 시신경 입원,그녀 당신 대통령 케네디 세계 남자 인간 전쟁 소설 무슨,호전 한의원 한약 택일 양방 약간 말기 현실 초기 필요
1,Topic0,Topic1,Topic2


###### LDA topic 분류별 data 수

In [107]:
pivot_table = pd.pivot_table(data=df_topics_score,
               values='sentences',
               index='dominant_topic_number',
               aggfunc='count')
pivot_table

,sentences
dominant_topic_number,
0,9309
1,2175
2,830
